VQEAC
=============================

In [8]:
import pennylane as qml
from pennylane import numpy as np

h2_dataset = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")
symbols = ['H', 'H']
coordinates = np.array([0,0,-0.371,0,0,0.371])
h2 = h2_dataset[0]

In [9]:
h2.hf_state

tensor([1, 1, 0, 0], dtype=int64, requires_grad=True)

In [10]:
qml.qchem.molecular_hamiltonian(symbols, coordinates)

(<Hamiltonian: terms=15, wires=[0, 1, 2, 3]>, 4)

In [12]:
qbuits = len(h2.hf_state)

## VQE
Based on Rayleigh–Ritz functional
Each VQE needs the following:
- An Ansatz
- Loss function

In [13]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.4602732973487343) [Z2]
+ (-0.46027329734873423) [Z3]
+ (0.236947529016103) [Z1]
+ (0.23694752901610303) [Z0]
+ (0.7735975819585081) [I0]
+ (0.14060026997649225) [Z0 Z2]
+ (0.14060026997649225) [Z1 Z3]
+ (0.18165504230123497) [Z0 Z3]
+ (0.18165504230123497) [Z1 Z2]
+ (0.18451830790068346) [Z0 Z1]
+ (0.19173391712932242) [Z2 Z3]
+ (-0.04105477232474271) [Y0 Y1 X2 X3]
+ (-0.04105477232474271) [X0 X1 Y2 Y3]
+ (0.04105477232474271) [Y0 X1 X2 Y3]
+ (0.04105477232474271) [X0 Y1 Y2 X3]


## Expected (ha) expected energy

Before any run, we can assume that the Jordan Wigner representation `[1 1 0 0]`, which is also equals to  has the lowest energy. Let's calculate that

In [23]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(state):
    qml.BasisState(state, wires = range(qbuits))
    for op in h2.vqe_gates:
        qml.apply(op)
    return qml.expval(h2.hamiltonian)

In [38]:
print(f" h2.hf_state}")
circuit(h2.hf_state)

[1 1 0 0]


array(-1.13637658)

In [20]:
h2.vqe_gates

[DoubleExcitation(0.27324054462951564, wires=[0, 1, 2, 3])]

Taking the superposition with themselves and the higher/lower energy level (excite/de-excite)

In [19]:
print(qml.draw(circuit)())

0: ─╭|Ψ⟩─╭G²(0.27)─┤ ╭<𝓗>
1: ─├|Ψ⟩─├G²(0.27)─┤ ├<𝓗>
2: ─├|Ψ⟩─├G²(0.27)─┤ ├<𝓗>
3: ─╰|Ψ⟩─╰G²(0.27)─┤ ╰<𝓗>


## Quantum circuit to find the ground state

Given 2 $H$ and 4 qubits, the HF is the superposition of the states
$\alpha\ket{1100}+\beta\ket{0011}$

[comment]: # ($\alpha\ket{110000}+\beta\ket{001100}+\gamma\ket{000011}$ this is H3)


In [ ]:
def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

The circuit above prepares trial states for the $\mathrm{H}_2$ molecule
of the form:

$$\vert \Psi(\theta) \rangle =
c_\mathrm{HF}(\theta) \vert 1100 \rangle
+ c_{0123}(\theta) \vert 0011 \rangle
+ c_{02}(\theta) \vert 0110 \rangle
+ c_{13}(\theta) \vert 1001 \rangle,$$

where the coefficients $c$ are functions of the variational parameters
$\theta = (\theta_1, \theta_2, \theta_3)$ to be optimized by the VQE
algorithm. Note that the prepared state $\vert \Psi(\theta) \rangle$ is
a superposition of the Hartree-Fock (HF) state with all possible single
and double excitations that preserve the spin projection of the HF
reference state.


Running the VQE simulation
==========================

Now we proceed to optimize the variational parameters. First, we define
the device, in this case a qubit simulator:


In [ ]:
@qml.qnode(dev, interface="autograd")
def circuit(params, obs, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
    qml.DoubleExcitation(params[1], wires=[0, 1, 4, 5])

    return qml.expval(obs)

As a reminder, we also built the total spin operator $\hat{S}^2$ for
which we can now define a function to compute its expectation value:


In [ ]:
@qml.qnode(dev, interface="autograd")
def S2_exp_value(params):
    circuit(params, wires=range(qubits))
    return qml.expval(S2)

The total spin $S$ of the trial state can be obtained from the
expectation value $\langle \hat{S}^2 \rangle$ as

$$S = -\frac{1}{2} + \sqrt{\frac{1}{4} + \langle \hat{S}^2 \rangle}.$$

We define a function to compute the total spin.


In [ ]:
def total_spin(params):
    return -0.5 + np.sqrt(1 / 4 + S2_exp_value(params))

Now, we proceed to minimize the cost function to find the ground state.
We define the classical optimizer and initialize the circuit parameters.


In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.8)
np.random.seed(0)  # for reproducibility
theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)
print(theta)

We carry out the optimization over a maximum of 100 steps, aiming to
reach a convergence tolerance of $10^{-6}$ for the value of the cost
function.


In [ ]:
max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy = cost_fn(theta)
    spin = total_spin(theta)

    conv = np.abs(energy - prev_energy)

    if n % 4 == 0:
        print(f"Step = {n}, Energy = {energy:.8f} Ha, S = {spin:.4f}")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy:.8f} Ha")
print("\n" f"Optimal value of the circuit parameters = {theta}")

As a result, we have estimated the lowest-energy state of the hydrogen
molecule with total spin $S = 0$ which corresponds to the ground state.

Finding the lowest-lying excited state with $S=1$
=================================================

In the last part of the tutorial, we will use VQE to find the
lowest-lying excited state of the hydrogen molecule with total spin
$S=1$. In this case, we use the
`~.pennylane.qchem.excitations`{.interpreted-text role="func"} function
to generate excitations whose total-spin projection differs by the
quantity `delta_sz=1` with respect to the Hartree-Fock state.


In [ ]:
singles, doubles = qml.qchem.excitations(electrons, qubits, delta_sz=1)
print(singles)
print(doubles)

For the $\mathrm{H}_2$ molecule in a minimal basis set there are no
double excitations, but only a spin-flip single excitation from qubit 1
to 2. In this case, the circuit will contain only one
`~.pennylane.SingleExcitation`{.interpreted-text role="class"}
operation. Additionally, as we want to probe the excited state of the
hydrogen molecule, we initialize the qubit register to the state
$\vert 0011 \rangle$.


In [ ]:
def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, np.flip(hf), singles, doubles)

This allows us to prepare trial states of the form

$$\vert \Psi(\theta) \rangle = c_{03}(\theta) \vert 0101 \rangle
+ c_{0123}(\theta) \vert 0011 \rangle,$$

where the first term $\vert 0101 \rangle$ encodes a spin-flip excitation
with $S_z=-1$ and the second term is a double excitation with $S_z=0$.
Since an eigenstate of the electronic Hamiltonian cannot contain a
superposition of states with different total-spin projections, the
double excitation coefficient should vanish as the VQE algorithm
minimizes the cost function. The optimized state will correspond to the
lowest-energy state with spin quantum numbers $S=1, S_z=-1$.

Now, we define the new functions to compute the expectation values of
the Hamiltonian and the total spin operator for the new circuit.


In [ ]:
@qml.qnode(dev, interface="autograd")
def cost_fn(params):
    circuit(params, wires=range(qubits))
    return qml.expval(H)


@qml.qnode(dev, interface="autograd")
def S2_exp_value(params):
    circuit(params, wires=range(qubits))
    return qml.expval(S2)

Finally, we generate the new set of initial parameters, and proceed with
the VQE algorithm to optimize the variational circuit.


In [ ]:
np.random.seed(0)
theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy = cost_fn(theta)
    spin = total_spin(theta)

    conv = np.abs(energy - prev_energy)

    if n % 4 == 0:
        print(f"Step = {n}, Energy = {energy:.8f} Ha, S = {spin:.4f}")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the energy = {energy:.8f} Ha")
print("\n" f"Optimal value of the circuit parameters = {theta}")

As expected, the VQE algorithms has found the lowest-energy state with
total spin $S=1$ which is an excited state of the hydrogen molecule.


### Running in a real quantum computer